In [79]:
# !pip install rdkit-pypi
# !git clone https://github.com/molecularsets/moses.git

import matplotlib as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

from rdkit import Chem


In [80]:
import csv

def load_smiles_from_csv(path, split_type='train'):
    '''
    Loads SMILES strings from a CSV file.

    Args:
        path (str): Path to the CSV file
        split_type (str): Split type ('train' or 'test')

    Returns:
        list: List of SMILES strings
    '''
    smiles = []
    with open(path, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            if row['SPLIT'].strip().lower() == split_type:
                smiles.append(row['SMILES'].strip())
    return smiles

In [81]:
'''
Functions are from the RNN model we have but not entirely sure where they would fit in the VAE
Currently working on implementation, the process_smiles function may help in creating valid molecules
'''

# Function to add start and end tokens
def process_smiles(smiles_list):
    return ["^" + s + "$" for s in smiles_list]

# Create character dictionaries including special tokens
def create_vocab(smiles_list):
    all_chars = sorted(list(set(''.join(smiles_list))))
    char2idx = {ch: i + 1 for i, ch in enumerate(all_chars)}
    char2idx[''] = 0  # Padding token
    idx2char = {i: ch for ch, i in char2idx.items()}
    return char2idx, idx2char, len(char2idx)

# Enhanced tokenization
def tokenize(smiles, char2idx):
    return [char2idx.get(c, 0) for c in smiles]  # Default to 0 if unknown

def detokenize(tokens, idx2char):
    return ''.join([idx2char.get(t, '') for t in tokens if t != 0])

In [82]:
def extract_unique_chars(smiles_list):
    '''
    Extracts unique characters from a list of SMILES strings.

    Args:
        smiles_list (list): List of SMILES strings

    Returns:
        list: List of unique characters
    '''
    unique_chars = set()
    for smiles in smiles_list:
        unique_chars.update(smiles.strip())
    return sorted(unique_chars)

def clean_smiles(smiles):
    '''
    Cleans a SMILES string by removing unwanted characters.

    Args:
        smiles (str): SMILES string

    Returns:
        str: Cleaned SMILES string
    '''
    # Remove unwanted metadata and special characters
    cleaned = smiles.split(',')[0].strip()
    cleaned = cleaned.replace('#', '')  # Remove '#' characters
    cleaned = cleaned.replace('$', '')  # Remove '$' characters
    cleaned = cleaned.replace('^', '')  # Remove '^' characters if present
    return cleaned

def decode_smiles(one_hot_tensor, idx_to_char):
    '''
    Decodes a one-hot encoded tensor back to SMILES.

    Args:
        one_hot_tensor (torch.Tensor): One-hot encoded tensor
    '''
    smiles = ''
    one_hot_tensor = one_hot_tensor.view(-1, len(idx_to_char))  # unflatten
    for row in one_hot_tensor:
        idx = row.argmax().item()
        smiles += idx_to_char[idx]
    return smiles.strip()

def verify_smiles(smiles):
  '''
  Verifies the validity of a SMILES string using RDKit.

  Args:
      smiles (str): SMILES string to verify

  Returns:
      bool: True if valid, False otherwise
  '''
  mol = Chem.MolFromSmiles(smiles)
  return mol is not None

In [83]:
# Generate a new molecule from VAE by sampling from the latent space
def generate_smiles(model, latent_dim, idx_to_char, max_length, vocab_size, temperature=1.0):
    '''
    Generates a new SMILES string by sampling from the VAE's latent space.

    Args:
        model (nn.Module): VAE model
        latent_dim (int): Dimension of the latent space
    '''
    z = torch.randn(1, latent_dim).to(model.fc1.weight.device)  # Ensure z is on the same device as the model
    with torch.no_grad():
        generated = model.decode(z)

    # Add postprocessing to convert to SMILES
    probs = F.softmax(generated.view(max_length, vocab_size) / temperature, dim=-1)

    # Sample the next character from the probability distribution
    generated_tokens_indices = torch.multinomial(probs, 1).cpu().numpy().flatten()

    # Iterate through indices to build the SMILES string
    generated_smiles = "".join([idx_to_char.get(i, "") for i in generated_tokens_indices])
    generated_smiles = generated_smiles.replace('^', '').replace('$', '')

    # Verification using rdkit
    is_valid = verify_smiles(generated_smiles)
    if is_valid:
      return generated_smiles
    else:
      return "INVALID"

    # return generated_smiles

In [84]:
# Dataset class for SMILES strings

# Contemplate Protein To Vector Encoding
class SMILESDataset(Dataset):
    def __init__(self, smiles_list, max_length=150, char_to_idx=None):
        '''
        Initializes the SMILESDataset with a list of SMILES strings.

        Args:
            smiles_list (list): List of SMILES strings
            max_length (int): Maximum length of the SMILES strings
            char_to_idx (dict): Character-to-index mapping

        The dataset will one-hot encode each character in a SMILES string to a fixed-size tensor of shape (max_length * vocab_size).
        If a SMILES string is shorter than max_length, it will be padded with zeros. If longer, it will be truncated.
        '''
        self.smiles_list = smiles_list
        self.max_length = max_length

        if char_to_idx is None:
            raise ValueError("Please provide a fixed character-to-index mapping")
            # self.char_to_idx, self.idx_to_char = build_vocabulary(smiles_list)
        else:
            self.char_to_idx = char_to_idx
            self.idx_to_char = {v: k for k, v in char_to_idx.items()}

        self.vocab_size = len(self.char_to_idx)

        original_count = len(smiles_list)
        filtered = []
        invalid_count = 0

        for s in smiles_list:
            s = s.strip()
            if all(c in self.char_to_idx for c in s):
                filtered.append(s)
            else:
                invalid_count += 1
        print(f"Total: {original_count}, Valid: {len(filtered)}, Invalid: {invalid_count}")
        self.smiles_list = filtered

    def __len__(self):
        '''
        Returns:
            int: Number of valid SMILES strings in the dataset
        '''

        return len(self.smiles_list)

    def __getitem__(self, idx):
        '''
        Fetches the encoded version of a SMILES string at a given index.

        Args:
            idx (int): Index of the SMILES string to retrieve

        Returns:
            torch.Tensor: One-hot encoded tensor of the SMILES string of shape (max_length * vocab_size)
        '''

        smiles = self.smiles_list[idx]

        # One-hot encode the SMILES string
        encoded = torch.zeros(self.max_length, self.vocab_size)
        for i, char in enumerate(smiles[:self.max_length]):
            encoded[i, self.char_to_idx[char]] = 1.0

        return encoded.view(-1) #Flatten into 1D tensor

In [85]:
class AffineCouplingLayer(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        if input_dim % 2 != 0:
            input_dim += 1
        
        self.net = nn.Sequential(
            nn.Linear(input_dim // 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim // 2 * 2)
        )
        
        # Initialize last layer with zeros for stable training
        nn.init.zeros_(self.net[-1].weight)
        nn.init.zeros_(self.net[-1].bias)

    def forward(self, x, reverse=False):
        x1, x2 = x.chunk(2, dim=1)
        
        # Get scaling and translation factors
        st = self.net(x1)
        s, t = st.chunk(2, dim=1)
        
        # Apply scaling with numerical stability
        scale_factor = 0.001
        s = torch.tanh(s) * scale_factor
        
        # Compute log determinant (only from the scaling part)
        log_det = torch.sum(s, dim=1)
        
        if reverse:
            # Inverse transformation
            x2 = (x2 - t) * torch.exp(-s)
            return torch.cat([x1, x2], dim=1), -log_det
        else:
            # Forward transformation
            x2 = x2 * torch.exp(s) + t
            return torch.cat([x1, x2], dim=1), log_det

class Flow(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super().__init__()
        self.input_dim = input_dim + (input_dim % 2)
        
        # Coupling layers without batch norm
        self.layers = nn.ModuleList([
            AffineCouplingLayer(self.input_dim, hidden_dim)
            for _ in range(num_layers)
        ])
        
        # Layer normalization instead of batch norm
        self.norms = nn.ModuleList([
            nn.LayerNorm(self.input_dim)
            for _ in range(num_layers)
        ])

    def forward(self, x, reverse=False):
        # Initialize log determinant
        log_det_total = torch.zeros(x.size(0), device=x.device)
        
        # Handle odd dimensions
        if x.size(1) % 2 != 0:
            x = F.pad(x, (0, 1), 'constant', 0)
        
        # Process through layers
        if reverse:
            for layer in reversed(self.layers):
                x, log_det = layer(x, reverse=True)
                log_det_total = log_det_total + log_det
        else:
            for layer in self.layers:
                x, log_det = layer(x, reverse=False)
                log_det_total = log_det_total + log_det
        
        return x, log_det_total

    def get_latent(self, x):
        """Generate latent representation"""
        z, _ = self.forward(x)
        return z

In [86]:
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim, vocab_size):
        super(VAE, self).__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim

        # Encoder
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc_mu = nn.Linear(256, latent_dim)
        self.fc_logvar = nn.Linear(256, latent_dim)

        # Decoder
        self.fc3 = nn.Linear(latent_dim, 256)
        self.fc4 = nn.Linear(256, input_dim)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc_mu(h1), self.fc_logvar(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, self.input_dim))
        z = self.reparameterize(mu, logvar)

        recon_x = self.decode(z)
        return recon_x, mu, logvar

In [87]:
def compute_vae_loss(recon_x, x, mu, logvar, dataset):
    """
    Compute VAE loss with dataset-specific vocabulary size.
    
    Args:
        recon_x: Reconstructed input from VAE
        x: Original input data
        mu: Mean from VAE encoder
        logvar: Log variance from VAE encoder
        dataset: Dataset object containing vocab_size and max_length
    """
    batch_size = x.size(0)
    vocab_size = dataset.vocab_size
    seq_len = dataset.max_length
    
    # Reshape tensors to match dataset dimensions
    x = x.view(batch_size, seq_len * vocab_size)
    recon_x = recon_x.view(batch_size, seq_len * vocab_size)
    
    # Reconstruction loss (BCE)
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    
    # KL divergence
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    # Weight the KLD term
    beta = 0.1  # Adjust this weight to balance reconstruction vs. KLD
    
    return BCE + beta * KLD

In [88]:
# Load SMILES strings
# with open('dataset/train.txt', 'r') as f:
#     smiles_train = [line.strip() for line in f][1:]

# with open('dataset/test.txt', 'r') as f:
#     smiles_test = [line.strip() for line in f]

smiles_train = load_smiles_from_csv('dataset/train.txt', split_type='train')
smiles_test = load_smiles_from_csv('dataset/test.txt', split_type='test')  # if test rows are in same file

# Apply cleaning to your SMILES
smiles_train = [clean_smiles(smiles) for smiles in smiles_train]
smiles_test = [clean_smiles(smiles) for smiles in smiles_test]
# smiles_train = process_smiles(smiles_train)
# smiles_test = process_smiles(smiles_test)
# smiles_train = smiles_train[:10000]
# smiles_test = smiles_test[:10000]

# print(f"Raw SMILES loaded: train={len(smiles_train)}, test={len(smiles_test)}") # output for testing purposes
all_smiles = smiles_train + smiles_test
unique_chars = extract_unique_chars(all_smiles)

print(f"Total unique characters: {len(unique_chars)}")
print("Unique characters in dataset:")
print(unique_chars)

# Use extracted unique characters to rebuild vocabulary
VALID_CHARS = unique_chars
char_to_idx = {c: i for i, c in enumerate(VALID_CHARS)}
idx_to_char = {i: c for c, i in char_to_idx.items()}

# Create datasets
train_dataset = SMILESDataset(smiles_train, max_length=150, char_to_idx=char_to_idx)
test_dataset = SMILESDataset(smiles_test, max_length=150, char_to_idx=char_to_idx)
print("Training Vocabulary Size:", train_dataset.vocab_size)
print("Test Vocabulary Size:", test_dataset.vocab_size) # Should be the same


print(f"# Train SMILES after filtering: {len(train_dataset)}")
print(f"# Test SMILES after filtering: {len(test_dataset)}")
# train_dataset = SMILESDataset(smiles_train)
# test_dataset = SMILESDataset(smiles_test, char_to_idx=train_dataset.char_to_idx)  # Share vocabulary

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)  # No need to shuffle test data

print(f"Number of batches in train_loader: {len(train_loader)}")
print(f"Number of batches in test_loader: {len(test_loader)}")

Total unique characters: 25
Unique characters in dataset:
['(', ')', '-', '1', '2', '3', '4', '5', '6', '=', 'B', 'C', 'F', 'H', 'N', 'O', 'S', '[', ']', 'c', 'l', 'n', 'o', 'r', 's']
Total: 1584663, Valid: 1584663, Invalid: 0
Total: 176074, Valid: 176074, Invalid: 0
Training Vocabulary Size: 25
Test Vocabulary Size: 25
# Train SMILES after filtering: 1584663
# Test SMILES after filtering: 176074
Number of batches in train_loader: 198083
Number of batches in test_loader: 22010


In [89]:
# Check a batch of data
for i, data in enumerate(train_loader):
    if i == 0:  # Just visualize the first batch
        print(data)
        break

# Visualize 3 samples
print("\nSample SMILES visualizations:")
for i in range(3):
    encoded = train_dataset[i]
    original = train_dataset.smiles_list[i]
    decoded = decode_smiles(encoded, train_dataset.idx_to_char)

    print(f"\nSample {i+1}")
    print(f"Original : {original}")
    print(f"Decoded  : {decoded}")
    print(f"Shape    : {encoded.shape}")

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

Sample SMILES visualizations:

Sample 1
Original : CCCS(=O)c1ccc2[nH]c(=NC(=O)OC)[nH]c2c1
Decoded  : CCCS(=O)c1ccc2[nH]c(=NC(=O)OC)[nH]c2c1((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((
Shape    : torch.Size([3750])

Sample 2
Original : CC(C)(C)C(=O)C(Oc1ccc(Cl)cc1)n1ccnc1
Decoded  : CC(C)(C)C(=O)C(Oc1ccc(Cl)cc1)n1ccnc1((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((
Shape    : torch.Size([3750])

Sample 3
Original : Cc1c(Cl)cccc1Nc1ncccc1C(=O)OCC(O)CO
Decoded  : Cc1c(Cl)cccc1Nc1ncccc1C(=O)OCC(O)CO((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((

In [90]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
torch._dynamo.config.suppress_errors = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(f"Using device: {device}")

# Instantiate the VAE model
input_dim = train_dataset.vocab_size * train_dataset.max_length  # Flatten the input (max_length x vocab_size)
latent_dim = 128

vocab_size = train_dataset.vocab_size
max_length = train_dataset.max_length

print("Vocab size:", train_dataset.vocab_size)
print("max_length:", train_dataset.max_length)
print("Input dim:", input_dim)

flow = Flow(input_dim = input_dim, hidden_dim=256, num_layers=4).to(device)
vae = VAE(input_dim, latent_dim, len(idx_to_char)).to(device)

# Optimizer
# Separate optimizers for Flow and VAE
flow_optimizer = torch.optim.Adam(flow.parameters(), lr=0.0001)
vae_optimizer = torch.optim.Adam(vae.parameters(), lr=0.0001)

# Training parameters
epochs = 100
early_stop_patience = 5
best_loss = float('inf')
patience_counter = 0
min_delta = 0.001

# Lists to track losses
flow_losses = []
vae_losses = []
total_losses = []


Using device: cuda
Vocab size: 25
max_length: 150
Input dim: 3750


In [91]:
class CombinedModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim, num_flow_layers):
        super().__init__()
        
        # Initialize FLOW for creating latent representation
        self.flow = Flow(
            input_dim=input_dim,
            hidden_dim=hidden_dim,
            num_layers=num_flow_layers
        )
        
        # Initialize VAE to work with FLOW's output
        self.vae = VAE(
            input_dim=input_dim,  # Flow preserves dimensionality
            latent_dim=latent_dim,
            vocab_size=vocab_size
        )

    def forward(self, x):
        # First pass through FLOW to get latent representation
        z_flow, ldj = self.flow(x)
        
        # Use FLOW's output as input to VAE
        recon_x, mu, logvar = self.vae(z_flow)
        
        return recon_x, mu, logvar, ldj, z_flow

def train_step(model, data, flow_optimizer, vae_optimizer, device):
    """Separated training step for FLOW and VAE"""
    data = data.to(device)
    
    # 1. Train FLOW
    flow_optimizer.zero_grad()
    z_flow, ldj = model.flow(data)
    flow_loss = -ldj.mean()  # Maximize log-likelihood
    flow_loss.backward()
    flow_optimizer.step()
    
    # 2. Train VAE using FLOW's output (detached)
    vae_optimizer.zero_grad()
    with torch.no_grad():
        z_flow, _ = model.flow(data)  # Get fresh flow output
    recon_batch, mu, logvar = model.vae(z_flow.detach())
    vae_loss = compute_vae_loss(recon_batch, data, mu, logvar, train_dataset)
    vae_loss.backward()
    vae_optimizer.step()
    
    return flow_loss.item(), vae_loss.item()

In [92]:
# Training loop
model = CombinedModel(
    input_dim=input_dim,
    hidden_dim=256,
    latent_dim=latent_dim,
    num_flow_layers=4
).to(device)

flow_optimizer = optim.Adam(model.flow.parameters(), lr=0.0001)
vae_optimizer = optim.Adam(model.vae.parameters(), lr=0.0001)

for epoch in range(epochs):
    model.train()
    epoch_flow_loss = 0
    epoch_vae_loss = 0
    
    for batch_idx, data in enumerate(train_loader):
        flow_loss, vae_loss = train_step(
            model, data, flow_optimizer, vae_optimizer, device
        )
        
        epoch_flow_loss += flow_loss
        epoch_vae_loss += vae_loss
        
        if (batch_idx + 1) % 100 == 0:
            print(f'Batch [{batch_idx + 1}/{len(train_loader)}] | '
                  f'Flow Loss: {flow_loss:.4f} | '
                  f'VAE Loss: {vae_loss:.4f}')

    # Generate molecules using the trained models
    with torch.no_grad():
        z = torch.randn(1, latent_dim).to(device)
        z_flow, _ = model.flow(z)
        generated = model.vae.decode(z_flow)

Batch [100/198083] | Flow Loss: -7.3042 | VAE Loss: 8470.0967
Batch [200/198083] | Flow Loss: -7.4689 | VAE Loss: 1408.8718
Batch [300/198083] | Flow Loss: -7.4943 | VAE Loss: 1129.9821
Batch [400/198083] | Flow Loss: -7.4985 | VAE Loss: 1106.0850
Batch [500/198083] | Flow Loss: -7.4982 | VAE Loss: 1098.4154
Batch [600/198083] | Flow Loss: -7.4996 | VAE Loss: 1129.1578


KeyboardInterrupt: 

Flow add, it created a significant drop in loss but then negative loss and break model. Possibly try to make the model stop before that happens of adjust the parameters with flow.

In [ ]:
# Generate a new molecule from VAE by sampling from the latent space
generated_smiles = generate_smiles(vae, latent_dim, train_dataset.idx_to_char, max_length, vocab_size)  # pass idx_to_char

print(f"Generated SMILES: {generated_smiles}")

Thoughts:

Potetnial invalidity due to:
Insufficient training
More data needed
Expansion of latent space

Problems recorded:

Enountered a problem where the model is taking a terabyte of data at once and breaking.
  For now adjusted the number of strings in the dataset
  Learned that the model is not properly breaking up the strings and just taking them whole.

Encountered trouble with model only printing carbon and negative learning value
  Learned that vocab was not correctly understood by the model
  Adjusted how the data was recorded and one-hot encoding.

VAE model original version had problems in learning and structure may be off.
  To keep it simple for now we used a simple designed VAE version but taken from GPT as a template.

Problem with FLOW use and negative Loss
  Jacobian is improperly recorded and used. Need to rework Flow Model
  Look into Kosaraju GLOW model as a proper reference